# Cosmos-Reason1 + FiftyOne: Full-Dataset Video Reasoning Pipeline

This notebook:

1. Loads the `pjramg/Safe_Unsafe_Test` dataset from Hugging Face into FiftyOne
2. Copies the first video sample to `assets/sample.mp4` (for reference/debugging)
3. Runs the Cosmos-Reason1 inference script (`scripts/inference2.py`) **for every video sample** in the dataset
4. Parses the reasoning and answer from the model output
5. Stores the results as metadata on each sample (both nested and flattened fields)
6. Launches the FiftyOne App so you can inspect the results

Assumptions:
- The Cosmos-Reason1 repo is cloned locally at `/home/paula/cosmos-reason1`
- The inference script is at `/home/paula/cosmos-reason1/scripts/inference2.py`
- You have a working GPU environment with CUDA available


In [ ]:
# 1. (Optional) Install dependencies – run this only if needed
# !pip install fiftyone fiftyone_huggingface
# !pip install 'accelerate>=1.10.1' 'pyyaml>=6.0.2' 'qwen-vl-utils>=0.0.11' rich \
#               'torch>=2.7.1' 'torchcodec>=0.6.0' 'transformers>=4.51.3' 'vllm>=0.10.1.1'

# If cosmos-reason1-utils is local, uncomment and adjust the path if needed:
# !pip install -e /home/paula/cosmos-reason1/cosmos_reason1_utils
# !pip install qwen-vl-utils
# !pip install transformers
# !pip install vllm


In [2]:
# 2. Load dataset and copy the first video to assets/sample.mp4
import fiftyone as fo
import fiftyone.utils.huggingface as fouh
import shutil
import pathlib

ROOT = pathlib.Path.cwd()
ASSETS = ROOT / "assets"
ASSETS.mkdir(exist_ok=True)

# NOTE: omit overwrite=True so existing annotations are preserved
#dataset = fouh.load_from_hub("pjramg/Safe_Unsafe_Test", persistent=True)
dataset = fo.load_dataset("pjramg/Safe_Unsafe_Test")

sample = dataset.first()

if sample is None:
    raise RuntimeError("Dataset is empty")
if sample.media_type != "video":
    raise RuntimeError(f"First sample is not a video (media_type={sample.media_type}). Use --images instead.")

dst = ASSETS / "sample.mp4"
shutil.copy2(sample.filepath, dst)
print("Reference video copied to:", dst)


Reference video copied to: /home/paula/notebooks/assets/sample.mp4


In [3]:
# 3. CUDA / environment sanity check
import torch
import os

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Current device index:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(0))
    print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))


CUDA available: True
CUDA device count: 1
Current device index: 0
Device name: NVIDIA L4
CUDA_VISIBLE_DEVICES: None


In [13]:
# 4. Helper to run Cosmos-Reason1 inference on a single video
import os
import sys          # 👈 NEW: use the current Python interpreter
import subprocess

# Make sure the alloc conf is visible to child processes
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
BASE_ENV = dict(os.environ)

COSMOS_SCRIPT = "/home/paula/cosmos-reason1/scripts/inference2.py"
PROMPT_FILE = "/home/paula/cosmos-reason1/prompts/question.yaml"
VISION_CONFIG = "/home/paula/cosmos-reason1/configs/vision_config.yaml"
QUESTION = "What are the potential safety hazards?"

def run_cosmos_on_video(video_path: str) -> str:
    """Runs Cosmos-Reason1 inference on the given video and returns full stdout text."""
    cmd = [
        sys.executable,  # 👈 use the same Python that's running this notebook
        COSMOS_SCRIPT,
        "--prompt", PROMPT_FILE,
        "--question", QUESTION,
        "--reasoning",
        "--videos", video_path,
        "-v",
        "--vision-config", VISION_CONFIG,
        "--gpu-memory-utilization", "0.90",
        "--max-model-len", "4096",
    ]

    print("\n======================================")
    print("Running Cosmos-Reason1 on:", video_path)
    print("Command:\n", " ".join(cmd), "\n")

    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        env=BASE_ENV,
    )
    stdout_lines = []
    for line in proc.stdout:
        stdout_lines.append(line)
        print(line, end="")  # stream to notebook
    proc.wait()

    stdout_text = "".join(stdout_lines)
    print("Inference complete for:", video_path)
    return stdout_text


In [14]:
# 5. Parsing helpers for Cosmos-Reason1 output
import re
import datetime as dt

def extract_last_assistant_block(text: str) -> str:
    """Extracts the last 'Assistant:' block from the Cosmos output."""
    blocks = re.split(r"\n-+\n", text)
    cands = [b for b in blocks if "Assistant:" in b]
    if not cands:
        return ""
    last = cands[-1]
    m = re.search(r"Assistant:\s*(.*)\Z", last, re.S)
    return m.group(1).strip() if m else ""

ANSI_RE = re.compile(r"\x1b\[[0-9;]*m")

def clean_ansi(s: str) -> str:
    """Remove ANSI escape codes from a string."""
    return ANSI_RE.sub("", s)


In [15]:
# 6. Run Cosmos-Reason1 on the WHOLE DATASET and store results in the FiftyOne DB
import tqdm

for idx, s in enumerate(dataset.iter_samples(progress=True), start=1):
    if s.media_type != "video":
        continue

    print(f"\n🔵 Processing sample {idx}/{len(dataset)}")
    print(f" - ID: {s.id}")
    print(f" - File: {s.filepath}\n")

    # If you want to skip already-processed samples, uncomment the block below:
    # if "cosmos_reason1_output" in s and s.cosmos_reason1_output.get("answer"):
    #     continue

    try:
        stdout_text = run_cosmos_on_video(s.filepath)
        assistant = extract_last_assistant_block(stdout_text)

        think = re.search(r"<think>(.*?)</think>", assistant, re.S | re.I)
        reasoning = think.group(1).strip() if think else None
        answer = re.sub(r"<think>.*?</think>", "", assistant, flags=re.S | re.I).strip()

        record = {
            "question": QUESTION,
            "model": "nvidia/Cosmos-Reason1-7B",
            "reasoning": reasoning,
            "answer": answer,
            "raw": assistant,
            "timestamp_utc": dt.datetime.utcnow().isoformat() + "Z",
        }

        # Store nested dict field
        s["cosmos_reason1_output"] = record

        # Tag the sample for easy filtering
        if "cosmos_reason1" not in s.tags:
            s.tags.append("cosmos_reason1")

        # Flatten into top-level fields for convenient filtering/searching in FiftyOne App
        for key, value in record.items():
            if value is None:
                continue
            if isinstance(value, str):
                value = clean_ansi(value)
            s[key] = value

        s.save()
    except Exception as e:
        print(f"Error processing sample {s.id}: {e}")
        # Optionally store the error in a field for inspection later
        s["cosmos_reason1_error"] = str(e)
        s.save()

print("Finished processing all samples.")


                                                                            
🔵 Processing sample 1/40
 - ID: 68963f5e70e9318697620fca
 - File: /home/paula/fiftyone/huggingface/hub/pjramg/Safe_Unsafe_Test/data/3_tr24.mp4


Running Cosmos-Reason1 on: /home/paula/fiftyone/huggingface/hub/pjramg/Safe_Unsafe_Test/data/3_tr24.mp4
Command:
 /home/paula/notebooks/notebooks_env/bin/python /home/paula/cosmos-reason1/scripts/inference2.py --prompt /home/paula/cosmos-reason1/prompts/question.yaml --question What are the potential safety hazards? --reasoning --videos /home/paula/fiftyone/huggingface/hub/pjramg/Safe_Unsafe_Test/data/3_tr24.mp4 -v --vision-config /home/paula/cosmos-reason1/configs/vision_config.yaml --gpu-memory-utilization 0.90 --max-model-len 4096 

VisionConfig(
│   fps=1,
│   total_pixels=800000
)
SamplingParams(
│   max_tokens=4096,
│   n=1,
│   temperature=0.01,
│   seed=42
)
[
│   {
│   │   'role': 'system',
│   │   'content': 'Always respond in English.\n\nYou are a helpful v

In [16]:
# 7. Launch the FiftyOne App and inspect results
session = fo.launch_app(
    dataset,
    port=5151,
    address="0.0.0.0",  # listen on all interfaces
    auto=False,
)
print("FiftyOne URL:", session.url)

# Example: verify some fields on the first sample
sample = dataset.first()
print("Sample ID:", sample.id)
print("Tags:", sample.tags)
print("Has cosmos_reason1_output:", "cosmos_reason1_output" in sample.field_names)

# Flattened fields (what we wrote as s["question"], s["answer"])
question_val = sample.question if "question" in sample.field_names else None
answer_val = sample.answer if "answer" in sample.field_names else None
print("Question (flattened):", question_val)
print("Answer (flattened):", answer_val)

# Nested dict field, if you want to peek inside cosmos_reason1_output too:
if "cosmos_reason1_output" in sample.field_names and sample.cosmos_reason1_output is not None:
    print("Nested question:", sample.cosmos_reason1_output.get("question", None))
    print("Nested answer:", sample.cosmos_reason1_output.get("answer", None))


Session launched. Run `session.show()` to open the App in a cell output.
FiftyOne URL: http://0.0.0.0:5151/
Sample ID: 68963f5e70e9318697620fca
Tags: ['train', 'cosmos_reason1']
Has cosmos_reason1_output: True
Question (flattened): What are the potential safety hazards?
Answer (flattened): <think>
  Okay, let's see. The user is asking about the potential safety hazards in the 
video shown. First, I need to recall the details from the video. The scene is an
industrial warehouse with machinery, storage racks, and a forklift moving 
around. There's a person walking in the background.

  So, safety hazards in such environments usually include things like moving 
machinery, poor visibility, tripping hazards, and unsafe behaviors. Let me break
it down.

  The forklift is moving quickly across the frame. Forklifts can be heavy and 
move quickly, which might not be safe if there are people nearby. If the 
operator isn't careful, they could hit someone or something. Also, the person 
walking in